In [68]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr # For Pearson correlation
from sklearn.metrics import r2_score # For R-squared scores

In [69]:
CLAPmodel = "laion/larger_clap_music"
#CLAPmodel = "laion/larger_clap_general"
#CLAPmodel = "laion/larger_clap_music_and_speech"
#CLAPmodel = "laion/clap-htsat-unfused"
#CLAPmodel = "laion/clap-htsat-fused"

In [70]:
# Load CLAP model + processor
model = ClapModel.from_pretrained(CLAPmodel)
processor = AutoProcessor.from_pretrained(CLAPmodel)

config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/776M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/776M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [71]:
print('total number of CLAP\'s parameters:', sum(p.numel() for p in model.parameters()))

total number of CLAP's parameters: 193913882


In [72]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('CLAP model size: {:.3f}MB'.format(size_all_mb))

CLAP model size: 740.294MB


In [73]:
#This section is to convert the uploaded zip files into folders
import zipfile

# Define the paths to your zip files
zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

# Define the directory where you want to extract the files
extract_dir = "/content/" # You can change this if you want to extract elsewhere

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract each zip file
for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Erro r: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


# Process audio

In [74]:
audio_stimuli = []
stimuli_path = "/content/Exp1/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

In [75]:
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)
audio_embeddings = model.get_audio_features(**inputs)

In [76]:
print(audio_embeddings.shape)

torch.Size([59, 512])


# Process text

In [77]:
"""
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = [tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = [tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake", "like"]

dimensional_captions_perceived = [tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = [tag for tag in dimensional_tags]
print(dimensional_captions_induced)

"""
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]

discrete_captions_perceived = ["I perceive this sound as " + tag for tag in discrete_tags]
print(discrete_captions_perceived)
discrete_captions_induced = ["This sound makes me feel " + tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake", "like"]

dimensional_captions_perceived = ["I perceive this sound as " + tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = ["This sound makes me feel " + tag for tag in dimensional_tags]
print(dimensional_captions_induced)


['I perceive this sound as happiness', 'I perceive this sound as sadness', 'I perceive this sound as anger', 'I perceive this sound as tenderness', 'I perceive this sound as fear']
['This sound makes me feel happiness', 'This sound makes me feel sadness', 'This sound makes me feel anger', 'This sound makes me feel tenderness', 'This sound makes me feel fear']
['I perceive this sound as positive', 'I perceive this sound as relaxed', 'I perceive this sound as awake', 'I perceive this sound as like']
['This sound makes me feel positive', 'This sound makes me feel relaxed', 'This sound makes me feel awake', 'This sound makes me feel like']


In [78]:
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

#For Dimensional Induced (IDim)
IDim_tag_inputs = processor(text=dimensional_captions_induced, return_tensors="pt", padding=True)
IDim_tag_embeds = model.get_text_features(**IDim_tag_inputs)

#For Dimensional Perceived (PDim)
PDim_tag_inputs = processor(text=dimensional_captions_perceived, return_tensors="pt", padding=True)
PDim_tag_embeds = model.get_text_features(**PDim_tag_inputs)

#For all of Dimensional
Dim_tag_embeds = (IDim_tag_embeds + PDim_tag_embeds)/2


#For Discrete Induced (IDisc)
IDisc_tag_inputs = processor(text=discrete_captions_induced, return_tensors="pt", padding=True)
IDisc_tag_embeds = model.get_text_features(**IDisc_tag_inputs)

#For Discrete Perceived (PDisc)
PDisc_tag_inputs = processor(text=discrete_captions_perceived, return_tensors="pt", padding=True)
PDisc_tag_embeds = model.get_text_features(**PDisc_tag_inputs)

#For all of Discrete
Disc_tag_embeds = (IDisc_tag_embeds + PDisc_tag_embeds)/2

# Generate outputs

In [79]:
#For Dimensional Induced (IDim)
print("IDim")
IDim_sims = torch.matmul(audio_embeddings, IDim_tag_embeds.T)
print(IDim_sims.shape) #torch.Size([59, 4])

#For Discrete Perceived (PDisc)
print("PDim")
PDim_sims = torch.matmul(audio_embeddings, PDim_tag_embeds.T)
print(PDim_sims.shape) #torch.Size([59, 4])

print("Dim")
#For all of Discrete
Dim_sims = torch.matmul(audio_embeddings, Dim_tag_embeds.T)
print(Dim_sims.shape) #torch.Size([59, 4])

#For Discrete Induced (IDisc)
print("IDisc")
IDisc_sims = torch.matmul(audio_embeddings, IDisc_tag_embeds.T)
print(IDisc_sims.shape) #torch.Size([59, 5])

#For Discrete Perceived (PDisc)
print("PDisc")
PDisc_sims = torch.matmul(audio_embeddings, PDisc_tag_embeds.T)
print(PDisc_sims.shape) #torch.Size([59, 5])

print("Disc")
#For all of Discrete
Disc_sims = torch.matmul(audio_embeddings, Disc_tag_embeds.T)
print(Disc_sims.shape) #torch.Size([59, 5])

IDim
torch.Size([59, 4])
PDim
torch.Size([59, 4])
Dim
torch.Size([59, 4])
IDisc
torch.Size([59, 5])
PDisc
torch.Size([59, 5])
Disc
torch.Size([59, 5])


## Load csv files and extract related columns

In [80]:
#For Dimensional
def GetDataDim(path, sims):
  IDim_path = path
  IDim_responses = []

  all_dfs = []

  for file in os.listdir(IDim_path):
      if file.endswith(".csv"):
          df = pd.read_csv(os.path.join(IDim_path, file))
          df.columns = df.columns.str.strip()
          required_columns = ['positive', 'relaxed', 'awake', 'like']
          if all(col in df.columns for col in required_columns):
              all_dfs.append(df)

  mean_positive_vector = []
  mean_relaxed_vector = []
  mean_awake_vector = []
  mean_like_vector = []

  num_rows = all_dfs[0].shape[0]

  for i in range(num_rows):
      current_row_positives = []
      current_row_relaxeds = []
      current_row_awakes = []
      current_row_likes = []

      # For the current row index 'i', collect values from all DataFrames
      for df in all_dfs:
          current_row_positives.append(df.iloc[i]['positive'])
          current_row_relaxeds.append(df.iloc[i]['relaxed'])
          current_row_awakes.append(df.iloc[i]['awake'])
          current_row_likes.append(df.iloc[i]['like'])

      # Calculate the mean for the current row across all files, for each column
      mean_positive_vector.append(np.mean(current_row_positives))
      mean_relaxed_vector.append(np.mean(current_row_relaxeds))
      mean_awake_vector.append(np.mean(current_row_awakes))
      mean_like_vector.append(np.mean(current_row_likes))

  IDim_responses = {
      'positive_vector': mean_positive_vector,
      'relaxed_vector': mean_relaxed_vector,
      'awake_vector': mean_awake_vector,
      'like_vector': mean_like_vector
  }

  print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
  print(f"Length of positive_vector: {len(IDim_responses['positive_vector'])}")
  print(f"Length of relaxed_vector: {len(IDim_responses['relaxed_vector'])}")
  print(f"Length of awake_vector: {len(IDim_responses['awake_vector'])}")
  print(f"Length of like_vector: {len(IDim_responses['like_vector'])}")

  scaled_sims = sims.detach().clone()

  for i in range(sims.shape[1]):
      sims_column = sims[:,i]
      # Find the min and max values in the current sims tensor
      old_min = sims_column.min()
      old_max = sims_column.max()

      # Define the new desired range
      new_min = 1.0
      new_max = 9.0

      # Apply the min-max scaling formula
      scaled_sims[:,i] = ((sims_column - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

  print(f"Scaled sims shape: {scaled_sims.shape}")
  print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
  print(f"Scaled sims min value: {scaled_sims.min():.4f}")
  print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

  human_ratings_tensor = torch.tensor([
      IDim_responses['positive_vector'],
      IDim_responses['relaxed_vector'],
      IDim_responses['awake_vector'],
      IDim_responses['like_vector']
  ], dtype=torch.float32).T # Transpose to get shape (59, 3)

  print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
  print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")

  return scaled_sims, human_ratings_tensor



#For Discrete
def GetDataDisc(path, sims):
  IDim_path = path
  IDim_responses = []

  all_dfs = []

  for file in os.listdir(IDim_path):
      if file.endswith(".csv"):
          df = pd.read_csv(os.path.join(IDim_path, file))
          df.columns = df.columns.str.strip()
          required_columns = ["happiness", "sadness", "anger", "tenderness", "fear"]
          if all(col in df.columns for col in required_columns):
              all_dfs.append(df)

  mean_happiness_vector = []
  mean_sadness_vector = []
  mean_anger_vector = []
  mean_tenderness_vector = []
  mean_fear_vector = []

  num_rows = all_dfs[0].shape[0]

  for i in range(num_rows):
      current_row_happiness = []
      current_row_sadness = []
      current_row_anger = []
      current_row_tenderness = []
      current_row_fear = []

      # For the current row index 'i', collect values from all DataFrames
      for df in all_dfs:
          current_row_happiness.append(df.iloc[i]['happiness'])
          current_row_sadness.append(df.iloc[i]['sadness'])
          current_row_anger.append(df.iloc[i]['anger'])
          current_row_tenderness.append(df.iloc[i]['tenderness'])
          current_row_fear.append(df.iloc[i]['fear'])

      # Calculate the mean for the current row across all files, for each column
      mean_happiness_vector.append(np.mean(current_row_happiness))
      mean_sadness_vector.append(np.mean(current_row_sadness))
      mean_anger_vector.append(np.mean(current_row_anger))
      mean_tenderness_vector.append(np.mean(current_row_tenderness))
      mean_fear_vector.append(np.mean(current_row_fear))

  IDim_responses = {
      'happiness_vector': mean_happiness_vector,
      'sadness_vector': mean_sadness_vector,
      'anger_vector': mean_anger_vector,
      'tenderness_vector': mean_tenderness_vector,
      'fear_vector': mean_fear_vector
  }

  print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
  print(f"Length of happiness_vector: {len(IDim_responses['happiness_vector'])}")
  print(f"Length of sadness_vector: {len(IDim_responses['sadness_vector'])}")
  print(f"Length of anger_vector: {len(IDim_responses['anger_vector'])}")
  print(f"Length of tiredness_vector: {len(IDim_responses['tenderness_vector'])}")
  print(f"Length of fear_vector: {len(IDim_responses['fear_vector'])}")

  scaled_sims = sims.detach().clone()

  for i in range(sims.shape[1]):
    sims_column = sims[:,i]
    # Find the min and max values in the current sims tensor
    old_min = sims_column.min()
    old_max = sims_column.max()

    # Define the new desired range
    new_min = 1.0
    new_max = 9.0

    # Apply the min-max scaling formula
    scaled_sims[:,i] = ((sims_column - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

  print(f"Scaled sims shape: {scaled_sims.shape}")
  print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
  print(f"Scaled sims min value: {scaled_sims.min():.4f}")
  print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

  human_ratings_tensor = torch.tensor([
      IDim_responses['happiness_vector'],
      IDim_responses['sadness_vector'],
      IDim_responses['anger_vector'],
      IDim_responses['tenderness_vector'],
      IDim_responses['fear_vector']
  ], dtype=torch.float32).T # Transpose to get shape (59, 3)

  print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
  print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")

  return scaled_sims, human_ratings_tensor


In [81]:
print("IDim")
IDim_scaled_sims, IDim_human_ratings_tensor = GetDataDim('/content/Exp1/Data/IDim', IDim_sims)
print("PDim")
PDim_scaled_sims, PDim_human_ratings_tensor = GetDataDim('/content/Exp1/Data/PDim', PDim_sims)

IDim

--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59
Length of like_vector: 59
Scaled sims shape: torch.Size([59, 4])
Scaled sims (first 5 rows):
tensor([[7.0522, 6.9745, 7.0027, 7.0362],
        [5.2657, 5.1823, 5.1666, 5.2407],
        [3.3214, 3.2529, 3.1849, 3.3019],
        [2.8025, 2.8878, 2.7934, 2.8338],
        [7.8589, 7.7201, 7.7511, 7.7760],
        [8.4106, 8.2147, 8.2391, 8.2861],
        [2.4958, 2.4913, 2.4657, 2.4267],
        [4.4350, 4.4796, 4.4195, 4.4297],
        [6.1052, 5.9869, 5.8877, 6.0390],
        [4.7573, 4.7840, 4.7033, 4.7567]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 4])
Human ratings tensor (first 5 rows):
tensor([[4.4872, 4.8798, 4.5185, 4.6463],
        [4.7682, 5.3057, 4.7414, 5.0326],
        [5.3105, 5.7312, 5.0760, 5.3854],
        [5.2582, 5.2651, 5.4488, 

In [82]:
print("IDisc")
IDisc_scaled_sims, IDisc_human_ratings_tensor = GetDataDisc('/content/Exp1/Data/IDisc', IDisc_sims)
print("PDisc")
PDisc_scaled_sims, PDisc_human_ratings_tensor = GetDataDisc('/content/Exp1/Data/PDisc', PDisc_sims)

IDisc

--- Mean Vectors (Mean across CSVs for each row position) ---
Length of happiness_vector: 59
Length of sadness_vector: 59
Length of anger_vector: 59
Length of tiredness_vector: 59
Length of fear_vector: 59
Scaled sims shape: torch.Size([59, 5])
Scaled sims (first 5 rows):
tensor([[7.0830, 7.0688, 7.0899, 7.0691, 7.0414],
        [5.2952, 5.2414, 5.2724, 5.3028, 5.2486],
        [3.3246, 3.2913, 3.3232, 3.3392, 3.2638],
        [2.8700, 2.8050, 2.7855, 2.7963, 2.8470],
        [7.8158, 7.7799, 7.8469, 7.8163, 7.7940],
        [8.3338, 8.2519, 8.4165, 8.3906, 8.3380],
        [2.4371, 2.4147, 2.4968, 2.4990, 2.4332],
        [4.4526, 4.3761, 4.4203, 4.4192, 4.4335],
        [6.0970, 6.0629, 6.1096, 6.1071, 6.0396],
        [4.7932, 4.7219, 4.7375, 4.7376, 4.7479]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 5])
Human ratings tensor (first 5 rows):
tensor([[1.8202, 3.3418, 2.7948, 1.9789, 4.143

In [83]:
#For Dimensional
def GetMergedDim(sims):
  # Load pre-aggregated mean vectors from aggregate_data.csv
  df = pd.read_csv('/content/Exp1/Data/aggregate_data.csv')  # This already includes mean IDim and PDim

  # Sort by StimNo to match audio alignment
  df = df.sort_values("StimNo").reset_index(drop=True)

  # Extract the mean dimensional responses
  Dim_responses = {
      'positive_vector': df['positive'].tolist(),
      'relaxed_vector': df['relaxed'].tolist(),
      'awake_vector': df['awake'].tolist(),
      'like_vector': df['like'].tolist()
  }

  print("\n--- Mean Vectors (from aggregate_data.csv) ---")
  print(f"Length of positive_vector: {len(Dim_responses['positive_vector'])}")
  print(f"Length of relaxed_vector: {len(Dim_responses['relaxed_vector'])}")
  print(f"Length of awake_vector: {len(Dim_responses['awake_vector'])}")
  print(f"Length of like_vector: {len(Dim_responses['like_vector'])}")

  scaled_sims = sims.detach().clone()

  for i in range(sims.shape[1]):
      sims_column = sims[:,i]
      # Find the min and max values in the current sims tensor
      old_min = sims_column.min()
      old_max = sims_column.max()

      # Define the new desired range
      new_min = 1.0
      new_max = 9.0

      # Apply the min-max scaling formula
      scaled_sims[:,i] = ((sims_column - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

  print(f"Scaled sims shape: {scaled_sims.shape}")
  print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
  print(f"Scaled sims min value: {scaled_sims.min():.4f}")
  print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

  human_ratings_tensor = torch.tensor([
      Dim_responses['positive_vector'],
      Dim_responses['relaxed_vector'],
      Dim_responses['awake_vector'],
      Dim_responses['like_vector']
  ], dtype=torch.float32).T # Transpose to get shape (59, 4)

  print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
  print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")

  return scaled_sims, human_ratings_tensor




#For Discrete
def GetMergedDisc(sims):
  #Extracting data from aggregate_data.csv to get answers for purely Discrete
  df = pd.read_csv('/content/Exp1/Data/aggregate_data.csv')

  # Sort by StimNo to match audio alignment
  df = df.sort_values("StimNo").reset_index(drop=True)

  # Extract the mean responses
  Dim_responses = {
      'happiness_vector': df['happiness'].tolist(),
      'sadness_vector': df['sadness'].tolist(),
      'anger_vector': df['anger'].tolist(),
      'tenderness_vector': df['tenderness'].tolist(),
      'fear_vector': df['fear'].tolist()
  }

  print("\n--- Mean Vectors (from aggregate_data.csv) ---")
  print(f"Length of happiness_vector: {len(Dim_responses['happiness_vector'])}")
  print(f"Length of sadness_vector: {len(Dim_responses['sadness_vector'])}")
  print(f"Length of anger_vector: {len(Dim_responses['anger_vector'])}")
  print(f"Length of tenderness_vector_vector: {len(Dim_responses['tenderness_vector'])}")
  print(f"Length of fear_vector: {len(Dim_responses['fear_vector'])}")

  scaled_sims = sims.detach().clone()

  for i in range(sims.shape[1]):
      sims_column = sims[:,i]
      # Find the min and max values in the current sims tensor
      old_min = sims_column.min()
      old_max = sims_column.max()

      # Define the new desired range
      new_min = 1.0
      new_max = 9.0

      # Apply the min-max scaling formula
      scaled_sims[:,i] = ((sims_column - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

  print(f"Scaled sims shape: {scaled_sims.shape}")
  print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
  print(f"Scaled sims min value: {scaled_sims.min():.4f}")
  print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

  human_ratings_tensor = torch.tensor([
      Dim_responses['happiness_vector'],
      Dim_responses['sadness_vector'],
      Dim_responses['anger_vector'],
      Dim_responses['tenderness_vector'],
      Dim_responses['fear_vector']
  ], dtype=torch.float32).T # Transpose to get shape (59, 5)

  print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
  print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")

  return scaled_sims, human_ratings_tensor

In [84]:
print("Dim")
Dim_scaled_sims, Dim_human_ratings_tensor = GetMergedDim(Dim_sims)
print("Disc")
Disc_scaled_sims, Disc_human_ratings_tensor = GetMergedDisc(Disc_sims)

Dim

--- Mean Vectors (from aggregate_data.csv) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59
Length of like_vector: 59
Scaled sims shape: torch.Size([59, 4])
Scaled sims (first 5 rows):
tensor([[7.0671, 6.9900, 7.0200, 7.0525],
        [5.2959, 5.2118, 5.1949, 5.2710],
        [3.3742, 3.3053, 3.2368, 3.3554],
        [2.8272, 2.9101, 2.8121, 2.8563],
        [7.8277, 7.6884, 7.7205, 7.7452],
        [8.3738, 8.1780, 8.2030, 8.2500],
        [2.4843, 2.4771, 2.4510, 2.4133],
        [4.4207, 4.4647, 4.4018, 4.4145],
        [6.1513, 6.0284, 5.9282, 6.0828],
        [4.7554, 4.7802, 4.6970, 4.7532]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 4])
Human ratings tensor (first 5 rows):
tensor([[3.9920, 4.7386, 3.9793, 4.7069],
        [4.4554, 5.1961, 3.9989, 5.0853],
        [4.9997, 5.6211, 4.6525, 5.2916],
        [5.2295, 5.3023, 5.1811, 5.1007],
       

# Evaluate

In [85]:
def format4(x):
    return "{:.4f}".format(float(x))

def format2(x):
    return "{:.2f}".format(float(x))

def format_tuple(t):
    return "(" + ", ".join(format4(x) for x in t) + ")"

def DisplayDimStats(Title, scaled_sims, human_ratings_tensor):
  # Comparison Method 1: Mean Absolute Error (MAE)
  # MAE measures the average magnitude of the errors in a set of predictions, without considering their direction.
  mae = torch.mean(torch.abs(scaled_sims - human_ratings_tensor))


  # Comparison Method 2: Pearson Correlation Coefficient (per column)
  # Pearson correlation measures the linear relationship between two sets of data.
  # We'll calculate it for each of the  columns.
  correlation_positive, _ = pearsonr(scaled_sims[:, 0].detach().numpy(), human_ratings_tensor[:, 0].numpy())
  correlation_relaxed, _ = pearsonr(scaled_sims[:, 1].detach().numpy(), human_ratings_tensor[:, 1].numpy())
  correlation_awake, _ = pearsonr(scaled_sims[:, 2].detach().numpy(), human_ratings_tensor[:, 2].numpy())
  correlation_like, _ = pearsonr(scaled_sims[:, 3].detach().numpy(), human_ratings_tensor[:, 3].numpy())

  correlation_scores = [correlation_positive, correlation_relaxed, correlation_awake, correlation_like]
  # Calculate the average correlation
  average_correlation = np.mean(correlation_scores)


  # Comparison Method 3: Mean Absolute Percentage Error (MAPE)
  # MAPE measures the accuracy of a forecasting method in terms of percentage.
  # Formula: MAPE = (1/n) * sum(|(Actual - Forecast) / Actual|) * 100%

  # Calculate the absolute percentage error for each element
  # Since human responses are between 1 and 9, division by zero is not a concern.
  absolute_percentage_error = torch.abs((human_ratings_tensor - scaled_sims) / human_ratings_tensor) * 100
  # Calculate the mean of these percentage errors
  mape = torch.mean(absolute_percentage_error)


  # Comparison Method 4: Root Mean Squared Error (RMSE)
  # RMSE measures the square root of the average of the squared differences between predicted and actual values.
  # It gives a relatively high weight to large errors.
  # Formula: RMSE = sqrt(mean((Actual - Forecast)^2))
  # Calculate the squared differences
  squared_differences = (human_ratings_tensor - scaled_sims)**2
  # Calculate the mean of the squared differences (Mean Squared Error - MSE)
  mse = torch.mean(squared_differences)
  # Calculate the square root to get RMSE
  rmse = torch.sqrt(mse)

  # Comparison Method 5: R-squared values (Coefficient of Determination)
  # R² measures how well the predicted values approximate the actual values.
  # It represents the proportion of variance in the dependent variable
  # that is predictable from the independent variable(s).
  #
  # Formula: R² = 1 - (Sum of Squared Residuals / Total Sum of Squares)
  #
  # Interpretation:
  # - R² = 1.0: Perfect prediction
  # - R² = 0.0: Predictions are as good as the mean of the actual values
  # - R² < 0.0: Predictions are worse than just using the mean
  r2_valence = r2_score(human_ratings_tensor[:, 0].numpy(), scaled_sims[:, 0].detach().numpy())
  r2_tension = r2_score(human_ratings_tensor[:, 1].numpy(), scaled_sims[:, 1].detach().numpy())
  r2_energy = r2_score(human_ratings_tensor[:, 2].numpy(), scaled_sims[:, 2].detach().numpy())
  r2_like = r2_score(human_ratings_tensor[:, 3].numpy(), scaled_sims[:, 3].detach().numpy())


  r2_scores = [r2_valence, r2_tension, r2_energy, r2_like]
  # Calculate the average R²
  average_r2 = np.mean(r2_scores)

  Disc_rows = [
    ("MAE", format4(mae)),
    ("Pearson Correlation Coefficient (Valence, Tension, Energy, Like)",
    format_tuple(correlation_scores)),
    ("Average Correlation", format4(average_correlation)),
    ("MAPE", f"{format2(mape)}%"),
    ("RMSE", format4(rmse)),
    ("R-Squared Scores (Valence, Tension, Energy, Like)",
    format_tuple(r2_scores)),
    ("R-Squared Average", format4(average_r2)),
  ]
  Disc_df = pd.DataFrame(Disc_rows, columns=[Title, "Statistics"])
  # Display with column borders, no row index
  style = [{"selector": "td, th", "props": [("border", "1px solid gray")]}]
  display(Disc_df.style.hide(axis="index").set_table_styles(style))


def DisplayDiscStats(Title, scaled_sims, human_ratings_tensor):
  # Comparison Method 1: Mean Absolute Error (MAE)
  # MAE measures the average magnitude of the errors in a set of predictions, without considering their direction.
  mae = torch.mean(torch.abs(scaled_sims - human_ratings_tensor))


  # Comparison Method 2: Pearson Correlation Coefficient (per column)
  # Pearson correlation measures the linear relationship between two sets of data.
  # We'll calculate it for each of the  columns.
  correlation_happiness, _ = pearsonr(scaled_sims[:, 0].detach().numpy(), human_ratings_tensor[:, 0].numpy())
  correlation_sadness, _ = pearsonr(scaled_sims[:, 1].detach().numpy(), human_ratings_tensor[:, 1].numpy())
  correlation_anger, _ = pearsonr(scaled_sims[:, 2].detach().numpy(), human_ratings_tensor[:, 2].numpy())
  correlation_tenderness, _ = pearsonr(scaled_sims[:, 3].detach().numpy(), human_ratings_tensor[:, 3].numpy())
  correlation_fear, _ = pearsonr(scaled_sims[:, 4].detach().numpy(), human_ratings_tensor[:, 4].numpy())

  correlation_scores = [correlation_happiness, correlation_sadness, correlation_anger, correlation_tenderness, correlation_fear]
  # Calculate the average correlation
  average_correlation = np.mean(correlation_scores)


  # Comparison Method 3: Mean Absolute Percentage Error (MAPE)
  # MAPE measures the accuracy of a forecasting method in terms of percentage.
  # Formula: MAPE = (1/n) * sum(|(Actual - Forecast) / Actual|) * 100%

  # Calculate the absolute percentage error for each element
  # Since human responses are between 1 and 9, division by zero is not a concern.
  absolute_percentage_error = torch.abs((human_ratings_tensor - scaled_sims) / human_ratings_tensor) * 100
  # Calculate the mean of these percentage errors
  mape = torch.mean(absolute_percentage_error)


  # Comparison Method 4: Root Mean Squared Error (RMSE)
  # RMSE measures the square root of the average of the squared differences between predicted and actual values.
  # It gives a relatively high weight to large errors.
  # Formula: RMSE = sqrt(mean((Actual - Forecast)^2))
  # Calculate the squared differences
  squared_differences = (human_ratings_tensor - scaled_sims)**2
  # Calculate the mean of the squared differences (Mean Squared Error - MSE)
  mse = torch.mean(squared_differences)
  # Calculate the square root to get RMSE
  rmse = torch.sqrt(mse)

  # Comparison Method 5: R-squared values (Coefficient of Determination)
  # R² measures how well the predicted values approximate the actual values.
  # It represents the proportion of variance in the dependent variable
  # that is predictable from the independent variable(s).
  #
  # Formula: R² = 1 - (Sum of Squared Residuals / Total Sum of Squares)
  #
  # Interpretation:
  # - R² = 1.0: Perfect prediction
  # - R² = 0.0: Predictions are as good as the mean of the actual values
  # - R² < 0.0: Predictions are worse than just using the mean
  r2_happiness = r2_score(human_ratings_tensor[:, 0].numpy(), scaled_sims[:, 0].detach().numpy())
  r2_sadness = r2_score(human_ratings_tensor[:, 1].numpy(), scaled_sims[:, 1].detach().numpy())
  r2_anger = r2_score(human_ratings_tensor[:, 2].numpy(), scaled_sims[:, 2].detach().numpy())
  r2_tenderness = r2_score(human_ratings_tensor[:, 3].numpy(), scaled_sims[:, 3].detach().numpy())
  r2_fear = r2_score(human_ratings_tensor[:, 4].numpy(), scaled_sims[:, 4].detach().numpy())

  r2_scores = [r2_happiness, r2_sadness, r2_anger, r2_tenderness, r2_fear]
  # Calculate the average R²
  average_r2 = np.mean(r2_scores)

  Disc_rows = [
    ("MAE", format4(mae)),
    ("Pearson Correlation Coefficient (Happiness, Sadness, Anger, Tenderness, Fear)",
    format_tuple(correlation_scores)),
    ("Average Correlation", format4(average_correlation)),
    ("MAPE", f"{format2(mape)}%"),
    ("RMSE", format4(rmse)),
    ("R-Squared Scores (Happiness, Sadness, Anger, Tenderness, Fear)",
    format_tuple(r2_scores)),
    ("R-Squared Average", format4(average_r2)),
  ]
  Disc_df = pd.DataFrame(Disc_rows, columns=[Title, "Statistics"])
  # Display with column borders, no row index
  style = [{"selector": "td, th", "props": [("border", "1px solid gray")]}]
  display(Disc_df.style.hide(axis="index").set_table_styles(style))

In [86]:
DisplayDimStats("Dimensional Induced (IDim)", IDim_scaled_sims, IDim_human_ratings_tensor)
print("\n" + "-"*80 + "\n")
DisplayDimStats("Dimensional Perceived (PDim)", PDim_scaled_sims, PDim_human_ratings_tensor)
print("\n" + "-"*80 + "\n")
DisplayDiscStats("Discrete Induced (IDisc)", IDisc_scaled_sims, IDisc_human_ratings_tensor)
print("\n" + "-"*80 + "\n")
DisplayDiscStats("Discrete Perceived (PDisc)", PDisc_scaled_sims, PDisc_human_ratings_tensor)
print("\n" + "-"*80 + "\n")
DisplayDimStats("Dimensional", Dim_scaled_sims, Dim_human_ratings_tensor)
print("\n" + "-"*80 + "\n")
DisplayDiscStats("Discrete", Disc_scaled_sims, Disc_human_ratings_tensor)

Dimensional Induced (IDim),Statistics
MAE,1.8332
"Pearson Correlation Coefficient (Valence, Tension, Energy, Like)","(0.4015, 0.3448, 0.0842, 0.3975)"
Average Correlation,0.3070
MAPE,41.57%
RMSE,2.2177
"R-Squared Scores (Valence, Tension, Energy, Like)","(-2.7715, -2.3798, -12.2052, -2.1769)"
R-Squared Average,-4.8833



--------------------------------------------------------------------------------



Dimensional Perceived (PDim),Statistics
MAE,1.8261
"Pearson Correlation Coefficient (Valence, Tension, Energy, Like)","(0.3893, 0.3634, 0.0251, 0.4696)"
Average Correlation,0.3118
MAPE,41.00%
RMSE,2.2347
"R-Squared Scores (Valence, Tension, Energy, Like)","(-2.3929, -2.5415, -4.3167, -1.7268)"
R-Squared Average,-2.7445



--------------------------------------------------------------------------------



Discrete Induced (IDisc),Statistics
MAE,3.3709
"Pearson Correlation Coefficient (Happiness, Sadness, Anger, Tenderness, Fear)","(0.4549, -0.4340, -0.3218, 0.3611, -0.1315)"
Average Correlation,-0.0143
MAPE,151.63%
RMSE,3.9640
"R-Squared Scores (Happiness, Sadness, Anger, Tenderness, Fear)","(-15.8406, -43.2573, -13.1794, -29.4636, -24.8437)"
R-Squared Average,-25.3169


...

Discrete Perceived (PDisc),Statistics
MAE,2.9986
"Pearson Correlation Coefficient (Happiness, Sadness, Anger, Tenderness, Fear)","(0.3447, -0.4720, -0.2168, 0.3270, -0.1895)"
Average Correlation,-0.0413
MAPE,119.01%
RMSE,3.5978
"R-Squared Scores (Happiness, Sadness, Anger, Tenderness, Fear)","(-9.3653, -10.4791, -9.4297, -12.1716, -8.7568)"
R-Squared Average,-10.0405



--------------------------------------------------------------------------------



Dimensional,Statistics
MAE,1.8189
"Pearson Correlation Coefficient (Valence, Tension, Energy, Like)","(0.4057, 0.3602, 0.0481, 0.4383)"
Average Correlation,0.3131
MAPE,40.60%
RMSE,2.2089
"R-Squared Scores (Valence, Tension, Energy, Like)","(-2.7006, -2.4113, -6.8652, -1.9809)"
R-Squared Average,-3.4895



--------------------------------------------------------------------------------



Discrete,Statistics
MAE,3.1663
"Pearson Correlation Coefficient (Happiness, Sadness, Anger, Tenderness, Fear)","(0.4013, -0.4715, -0.2859, 0.3453, -0.1689)"
Average Correlation,-0.0360
MAPE,132.00%
RMSE,3.7618
"R-Squared Scores (Happiness, Sadness, Anger, Tenderness, Fear)","(-12.4294, -20.8635, -12.7148, -18.9315, -14.7811)"
R-Squared Average,-15.9440
